In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)
plt.rcParams["figure.figsize"] = (24, 12)

In [2]:
# get currently available data
df = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv", parse_dates=["date"])
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,0.126,0.126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,0.126,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,39835428.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [3]:
# select attributes that we find interesting
attributes = [
    "iso_code", "continent", "location", "date", "total_cases", "new_cases", "total_deaths", "new_deaths", "reproduction_rate", "icu_patients", "hosp_patients",
    "weekly_icu_admissions", "weekly_hosp_admissions", "new_tests", "total_tests", "positive_rate", "total_vaccinations", "people_vaccinated", "people_fully_vaccinated",
    "new_vaccinations", "stringency_index", "population", "population_density", "median_age", "aged_65_older", "aged_70_older", "gdp_per_capita", "extreme_poverty",
    "cardiovasc_death_rate", "diabetes_prevalence", "female_smokers", "male_smokers", "handwashing_facilities", "hospital_beds_per_thousand", "life_expectancy",
    "human_development_index", "excess_mortality"
]

df = df[attributes]

In [4]:
# now check which columns contain NaN values
df.isna().any().to_frame().T

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,icu_patients,hosp_patients,weekly_icu_admissions,weekly_hosp_admissions,new_tests,total_tests,positive_rate,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,False,True,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [5]:
# it's weird that continent contains NaNs and location doesn't, so let's take a look at that
print("First 5 rows that have NaN for the column continent")
df[df["continent"].isna()].head()

First 5 rows that have NaN for the column continent


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,icu_patients,hosp_patients,weekly_icu_admissions,weekly_hosp_admissions,new_tests,total_tests,positive_rate,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
623,OWID_AFR,NaN,Africa,2020-02-13,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.373486e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
624,OWID_AFR,NaN,Africa,2020-02-14,1.0,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.373486e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
625,OWID_AFR,NaN,Africa,2020-02-15,1.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.373486e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
626,OWID_AFR,NaN,Africa,2020-02-16,1.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.373486e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
627,OWID_AFR,NaN,Africa,2020-02-17,1.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.373486e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# let's store these rows which combine data of a continent in a seperate dataframe, so it doesn't give us any weird mistakes later/confuse us
# before we do this we should be sure that really only these combined rows have NaNs
print("Unique locations where continent is NaN", df[df["continent"].isna()]["location"].unique())
# looking good, so lets create a new dataframe
continent_df = df[df["continent"].isna()]
# drop these rows from the original dataframe
df = df[~df["continent"].isna()]

Unique locations where continent is NaN ['Africa' 'Asia' 'Europe' 'European Union' 'High income' 'International'
 'Low income' 'Lower middle income' 'North America' 'Oceania'
 'South America' 'Upper middle income' 'World']


In [7]:
# we would expect that new_cases should be almost complete (because it's the most important attribute), so let's take a look at that
print("Percentage of non-missing values for each country in the column new_cases:")
(df[["location", "new_cases"]].groupby("location").count()["new_cases"] / df[["location", "new_cases"]].groupby("location").size()).to_frame().T

Percentage of non-missing values for each country in the column new_cases:


location,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bermuda,Bhutan,Bolivia,Bonaire Sint Eustatius and Saba,Bosnia and Herzegovina,Botswana,Brazil,British Virgin Islands,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Cayman Islands,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,Cook Islands,Costa Rica,Cote d'Ivoire,Croatia,Cuba,Curacao,Cyprus,Czechia,Democratic Republic of Congo,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Eritrea,Estonia,Eswatini,Ethiopia,Faeroe Islands,Falkland Islands,Fiji,Finland,France,French Polynesia,Gabon,Gambia,Georgia,Germany,Ghana,Gibraltar,Greece,Greenland,Grenada,Guatemala,Guernsey,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Isle of Man,Israel,Italy,Jamaica,Japan,Jersey,Jordan,Kazakhstan,Kenya,Kiribati,Kosovo,Kuwait,Kyrgyzstan,Laos,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,Lithuania,Luxembourg,Macao,Madagascar,Malawi,Malaysia,Maldives,Mali,Malta,Marshall Islands,Mauritania,Mauritius,Mexico,Micronesia (country),Moldova,Monaco,Mongolia,Montenegro,Montserrat,Morocco,Mozambique,Myanmar,Namibia,Nauru,Nepal,Netherlands,New Caledonia,New Zealand,Nicaragua,Niger,Nigeria,Niue,North Macedonia,Northern Cyprus,Norway,Oman,Pakistan,Palau,Palestine,Panama,Papua New Guinea,Paraguay,Peru,Philippines,Pitcairn,Poland,Portugal,Qatar,Romania,Russia,Rwanda,Saint Helena,Saint Kitts and Nevis,Saint Lucia,Saint Vincent and the Grenadines,Samoa,San Marino,Sao Tome and Principe,Saudi Arabia,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Sint Maarten (Dutch part),Slovakia,Slovenia,Solomon Islands,Somalia,South Africa,South Korea,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan,Tajikistan,Tanzania,Thailand,Timor,Togo,Tokelau,Tonga,Trinidad and Tobago,Tunisia,Turkey,Turkmenistan,Turks and Caicos Islands,Tuvalu,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Vatican,Venezuela,Vietnam,Wallis and Futuna,Yemen,Zambia,Zimbabwe
0,1.0,0.9791,1.0,1.0,1.0,0.0,1.0,0.908419,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.99187,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.99839,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.998476,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.996721,0.988782,1.0,0.96124,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.962733,1.0,1.0,0.0,0.0,0.924383,1.0,0.998471,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.988854,0.0,1.0,0.95873,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.998374,0.996779,1.0,1.0,1.0,1.0,1.0,0.0,0.998405,1.0,1.0,0.998476,0.0,1.0,1.0,0.988562,1.0,1.0,1.0,1.0,1.0,0.9856,1.0,1.0,1.0,0.967427,1.0,1.0,0.991974,0.0,1.0,0.993209,0.998471,1.0,1.0,0.996737,1.0,1.0,1.0,0.914328,1.0,1.0,1.0,1.0,1.0,0.0,0.9625,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.998358,1.0,0.998363,0.903988,1.0,0.0,0.987138,0.980892,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.995153,0.985507,1.0,1.0,1.0,0.0,1.0,0.950388,1.0,1.0,0.957812,0.996956,1.0,1.0,1.0,0.9967,1.0,1.0,1.0,1.0,0.989426,1.0,1.0,0.97181,1.0,0.996743,0.0,0.049505,1.0,1.0,0.998353,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.998476,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0


In [8]:
# it seems that there are countries that don't have any values for new_cases or simply not enough values, these are obviously useless to us
# lets drop all countries that have missing values for more than half of their entries for the column new_cases 
# (if they have more than half of the values, then we can fix the missing values with interpolation later)
temp = ((df[["location", "new_cases"]].groupby("location").count()["new_cases"] / df[["location", "new_cases"]].groupby("location").size()) >= 0.5)
temp = temp[temp == True].index.tolist()
# drop all rows that don't fulfill the above defined criteria
df = df[df["location"].isin(temp)]
# reset index, so that it is correct again (we dropped rows)
df.reset_index(inplace = True, drop = True)

In [9]:
# number of trailing NaNs for column new_cases
temp = df.copy()
temp.reset_index(inplace = True)
last_index = temp.groupby("location").apply(lambda x: x.iloc[-1]["index"])
last_valid_index = temp.groupby("location").apply(lambda x: x["new_cases"].last_valid_index())
(last_index - last_valid_index).to_frame().T

location,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,Costa Rica,Cote d'Ivoire,Croatia,Cuba,Cyprus,Czechia,Democratic Republic of Congo,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Eritrea,Estonia,Eswatini,Ethiopia,Fiji,Finland,France,Gabon,Gambia,Georgia,Germany,Ghana,Greece,Grenada,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Israel,Italy,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,Kosovo,Kuwait,Kyrgyzstan,Laos,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,Lithuania,Luxembourg,Madagascar,Malawi,Malaysia,Maldives,Mali,Malta,Marshall Islands,Mauritania,Mauritius,Mexico,Micronesia (country),Moldova,Monaco,Mongolia,Montenegro,Morocco,Mozambique,Myanmar,Namibia,Nepal,Netherlands,New Zealand,Nicaragua,Niger,Nigeria,North Macedonia,Norway,Oman,Pakistan,Palau,Palestine,Panama,Papua New Guinea,Paraguay,Peru,Philippines,Poland,Portugal,Qatar,Romania,Russia,Rwanda,Saint Kitts and Nevis,Saint Lucia,Saint Vincent and the Grenadines,Samoa,San Marino,Sao Tome and Principe,Saudi Arabia,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Slovakia,Slovenia,Solomon Islands,Somalia,South Africa,South Korea,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan,Tajikistan,Tanzania,Thailand,Timor,Togo,Trinidad and Tobago,Tunisia,Turkey,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Vatican,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
# also while looking through the records, we found that some countries have leading NaNs for new_cases
# let's remove these (while we're at it, let's also remove trailing NaNs)
df = df.sort_values(by = ["location", "date"]).reset_index(drop = True)
# get the first and last valid index
first_valid_index = df.groupby("location").apply(lambda x: x["new_cases"].first_valid_index())
last_valid_index = df.groupby("location").apply(lambda x: x["new_cases"].last_valid_index())
# create list of indices that we want to keep
valid_indices = [np.arange(first, last+1) for first, last in zip(first_valid_index, last_valid_index)]
# flatten it to be a 1D array instead of 2D
valid_indices = [elem for sublist in valid_indices for elem in sublist]
df = df[df.index.isin(valid_indices)]
# we removed rows, so we need to reset the index
df.reset_index(drop = True, inplace = True)

In [11]:
# let's look at what percentage of values is still NaN for each column
print("Percentage of missing values for each column")
(df.isna().sum() / len(df)).to_frame().T

Percentage of missing values for each column


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,icu_patients,hosp_patients,weekly_icu_admissions,weekly_hosp_admissions,new_tests,total_tests,positive_rate,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,0.0,0.0,0.0,0.0,0.0,0.000026,0.093542,0.093568,0.109754,0.864297,0.840888,0.989169,0.981791,0.530422,0.528958,0.464266,0.763971,0.775826,0.79958,0.803945,0.095376,0.0,0.02105,0.045845,0.056616,0.051114,0.04235,0.352334,0.037813,0.02124,0.248762,0.259257,0.515019,0.123451,0.0052,0.031933,0.960853


In [12]:
# for some reason total_cases is complete, but new_cases isn't so let's fix that real quick
miss_indices = df[df["new_cases"].isna()].index
df.loc[miss_indices, "new_cases"] = list(df.iloc[miss_indices+1]["total_cases"] - np.array(df.iloc[miss_indices]["total_cases"]))

In [13]:
# also it seems that there are some columns where we simply have too many missing values for them to be useful, let's remove these
cols_to_drop = ["icu_patients", "hosp_patients", "weekly_icu_admissions", "weekly_hosp_admissions", "excess_mortality"]
df.drop(columns = cols_to_drop, inplace = True)

In [14]:
# let's try and fill the missing values for the remaining columns
# vaccinations numbers are very interesting to us so let's take a look at it
# for each country get the percentage of values that are not NaN for total_vaccinations
print("Percentage of non-NaN values for each country for the column total_vaccinations")
(df.groupby("location").count()["total_vaccinations"] / df.groupby("location").size()).to_frame().T

Percentage of non-NaN values for each country for the column total_vaccinations


location,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,Costa Rica,Cote d'Ivoire,Croatia,Cuba,Cyprus,Czechia,Democratic Republic of Congo,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Eritrea,Estonia,Eswatini,Ethiopia,Fiji,Finland,France,Gabon,Gambia,Georgia,Germany,Ghana,Greece,Grenada,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Israel,Italy,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,Kosovo,Kuwait,Kyrgyzstan,Laos,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,Lithuania,Luxembourg,Madagascar,Malawi,Malaysia,Maldives,Mali,Malta,Marshall Islands,Mauritania,Mauritius,Mexico,Micronesia (country),Moldova,Monaco,Mongolia,Montenegro,Morocco,Mozambique,Myanmar,Namibia,Nepal,Netherlands,New Zealand,Nicaragua,Niger,Nigeria,North Macedonia,Norway,Oman,Pakistan,Palau,Palestine,Panama,Papua New Guinea,Paraguay,Peru,Philippines,Poland,Portugal,Qatar,Romania,Russia,Rwanda,Saint Kitts and Nevis,Saint Lucia,Saint Vincent and the Grenadines,Samoa,San Marino,Sao Tome and Principe,Saudi Arabia,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Slovakia,Slovenia,Solomon Islands,Somalia,South Africa,South Korea,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan,Tajikistan,Tanzania,Thailand,Timor,Togo,Trinidad and Tobago,Tunisia,Turkey,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Vatican,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
0,0.046549,0.328407,0.017685,0.051948,0.065217,0.115702,0.469919,0.038898,0.391104,0.506431,0.34684,0.059801,0.430177,0.3,0.402662,0.042003,0.485226,0.173109,0.024917,0.153595,0.268534,0.029364,0.079932,0.386473,0.210181,0.481967,0.024671,0.011925,0.40553,0.063725,0.504601,0.060201,0.028192,0.043406,0.50641,0.369466,0.344771,0.037702,0.038143,0.076797,0.3229,0.368167,0.255776,0.202303,0.512156,0.047776,0.479032,0.023333,0.083893,0.350081,0.450567,0.07425,0.220367,0.092869,0.0,0.508065,0.06457,0.292562,0.050083,0.392912,0.478593,0.06457,0.043261,0.227053,0.485407,0.039735,0.476651,0.075503,0.387417,0.175207,0.023609,0.113861,0.068562,0.079077,0.39542,0.224756,0.174475,0.441358,0.413961,0.071656,0.048154,0.501618,0.517572,0.488408,0.110379,0.322137,0.235772,0.317355,0.27438,0.045977,0.188742,0.020867,0.185,0.117845,0.534091,0.392971,0.020221,0.02995,0.06229,0.519544,0.511327,0.420712,0.026756,0.275214,0.393568,0.409836,0.047218,0.463175,0.0,0.062914,0.045,0.466882,0.0,0.254098,0.040453,0.3125,0.384359,0.318182,0.041946,0.028765,0.22351,0.220521,0.364516,0.418417,0.028381,0.026756,0.12601,0.223833,0.544283,0.120385,0.165595,0.0,0.244698,0.389803,0.036789,0.332787,0.444444,0.240741,0.421824,0.423701,0.360841,0.494364,0.355487,0.092715,0.089376,0.192053,0.087748,0.067606,0.160194,0.049914,0.423701,0.160714,0.28268,0.116086,0.040886,0.189313,0.498366,0.513866,0.040816,0.066445,0.292007,0.389313,0.054983,0.335913,0.365591,0.036424,0.294702,0.075851,0.512862,0.02349,0.290076,0.068345,0.009967,0.317557,0.045302,0.029412,0.349338,0.311075,0.489292,0.177554,0.417886,0.429892,0.465224,0.503817,0.419835,0.117745,0.049587,0.0,0.031457,0.329771,0.02253,0.195,0.401338


In [15]:
# maybe the high number of missing values comes from leading NaNs? let's check that (those are not the number of leading NaNs!!!!!!!)
temp = df.copy()
temp.reset_index(inplace = True)
first_index = temp.groupby("location").apply(lambda x: x.iloc[0]["index"])
first_valid_index = temp.groupby("location").apply(lambda x: x["total_vaccinations"].first_valid_index())
print("Number of leading NaNs for each country for the column total_vaccinations")
(first_valid_index - first_index).to_frame().T

Number of leading NaNs for each country for the column total_vaccinations


location,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,Costa Rica,Cote d'Ivoire,Croatia,Cuba,Cyprus,Czechia,Democratic Republic of Congo,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Eritrea,Estonia,Eswatini,Ethiopia,Fiji,Finland,France,Gabon,Gambia,Georgia,Germany,Ghana,Greece,Grenada,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Israel,Italy,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,Kosovo,Kuwait,Kyrgyzstan,Laos,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,Lithuania,Luxembourg,Madagascar,Malawi,Malaysia,Maldives,Mali,Malta,Marshall Islands,Mauritania,Mauritius,Mexico,Micronesia (country),Moldova,Monaco,Mongolia,Montenegro,Morocco,Mozambique,Myanmar,Namibia,Nepal,Netherlands,New Zealand,Nicaragua,Niger,Nigeria,North Macedonia,Norway,Oman,Pakistan,Palau,Palestine,Panama,Papua New Guinea,Paraguay,Peru,Philippines,Poland,Portugal,Qatar,Romania,Russia,Rwanda,Saint Kitts and Nevis,Saint Lucia,Saint Vincent and the Grenadines,Samoa,San Marino,Sao Tome and Principe,Saudi Arabia,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Slovakia,Slovenia,Solomon Islands,Somalia,South Africa,South Korea,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan,Tajikistan,Tanzania,Thailand,Timor,Togo,Trinidad and Tobago,Tunisia,Turkey,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Vatican,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
0,364.0,307.0,339.0,329.0,346.0,340.0,301.0,395.0,392.0,306.0,322.0,362.0,303.0,324.0,335.0,304.0,328.0,342.0,422.0,385.0,323.0,343.0,360.0,325.0,389.0,296.0,449.0,567.0,379.0,401.0,323.0,363.0,423.0,448.0,305.0,327.0,348.0,356.0,395.0,293.0,355.0,309.0,445.0,302.0,301.0,403.0,278.0,395.0,326.0,351.0,325.0,345.0,335.0,366.0,NaN,304.0,381.0,391.0,363.0,337.0,338.0,373.0,357.0,382.0,335.0,352.0,306.0,326.0,348.0,383.0,413.0,336.0,483.0,354.0,396.0,299.0,306.0,351.0,316.0,355.0,371.0,303.0,302.0,331.0,363.0,391.0,315.0,324.0,356.0,14.0,379.0,308.0,375.0,358.0,277.0,358.0,300.0,421.0,389.0,292.0,302.0,303.0,418.0,349.0,396.0,331.0,370.0,316.0,NaN,377.0,313.0,300.0,NaN,362.0,305.0,349.0,340.0,332.0,350.0,305.0,369.0,367.0,314.0,356.0,347.0,373.0,370.0,356.0,280.0,308.0,343.0,NaN,390.0,316.0,375.0,351.0,339.0,395.0,299.0,300.0,297.0,305.0,319.0,338.0,334.0,339.0,368.0,167.0,364.0,343.0,310.0,357.0,308.0,300.0,348.0,354.0,304.0,297.0,162.0,397.0,348.0,400.0,365.0,338.0,367.0,361.0,345.0,330.0,300.0,344.0,423.0,371.0,510.0,401.0,388.0,368.0,338.0,373.0,308.0,353.0,358.0,342.0,345.0,325.0,351.0,381.0,203.0,NaN,340.0,409.0,394.0,392.0,335.0


In [16]:
# so it seems that there are a lot of leading NaNs, however for some countries we don't have any vaccination numbers, lets remove these countries
temp = (first_valid_index - first_index)
print("Number of countries for which we don't have any vaccination numbers: {}".format(len(temp[temp.isna()])))
# also after removing these countries, we need to reset the index (because we dropped some rows)
df = df[~df["location"].isin(temp[temp.isna()].index)].reset_index(drop = True)

Number of countries for which we don't have any vaccination numbers: 5


In [17]:
# let's look at the first value of total_vaccinations for each country that isn't NaN
first_valid_index = df.groupby("location").apply(lambda x: x["total_vaccinations"].first_valid_index())
print("First non-NaN value for each country for the column total_vaccinations")
df.iloc[list(first_valid_index)][["total_vaccinations", "location"]].set_index("location").T

First non-NaN value for each country for the column total_vaccinations


location,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,Costa Rica,Cote d'Ivoire,Croatia,Cuba,Cyprus,Czechia,Democratic Republic of Congo,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Estonia,Eswatini,Ethiopia,Fiji,Finland,France,Gabon,Gambia,Georgia,Germany,Ghana,Greece,Grenada,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Israel,Italy,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,Kosovo,Kuwait,Kyrgyzstan,Laos,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,Lithuania,Luxembourg,Madagascar,Malawi,Malaysia,Maldives,Mali,Malta,Mauritania,Mauritius,Mexico,Moldova,Monaco,Mongolia,Montenegro,Morocco,Mozambique,Myanmar,Namibia,Nepal,Netherlands,New Zealand,Nicaragua,Niger,Nigeria,North Macedonia,Norway,Oman,Pakistan,Palestine,Panama,Papua New Guinea,Paraguay,Peru,Philippines,Poland,Portugal,Qatar,Romania,Russia,Rwanda,Saint Kitts and Nevis,Saint Lucia,Saint Vincent and the Grenadines,Samoa,San Marino,Sao Tome and Principe,Saudi Arabia,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Slovakia,Slovenia,Solomon Islands,Somalia,South Africa,South Korea,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan,Tajikistan,Tanzania,Thailand,Timor,Togo,Trinidad and Tobago,Tunisia,Turkey,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
total_vaccinations,0.0,0.0,0.0,576.0,0.0,0.0,700.0,565.0,20.0,752.0,0.0,0.0,38965.0,0.0,4490.0,0.0,329.0,0.0,10683.0,0.0,0.0,0.0,0.0,0.0,0.0,1719.0,200.0,81.0,0.0,0.0,5.0,0.0,667.0,5324.0,420.0,1500000.0,18.0,13440.0,14297.0,55.0,0.0,7864.0,2037745.0,3901.0,1281.0,0.0,1.0,10246.0,0.0,0.0,17.0,0.0,0.0,6565.0,191.0,0.0,430000.0,0.0,1767.0,432.0,0.0,0.0,0.0,24355.0,0.0,447.0,0.0,0.0,62645.0,5889.0,0.0,38.0,2684.0,47.0,1094.0,4875.0,0.0,0.0,0.0,0.0,86.0,64.0,7336.0,0.0,125.0,0.0,0.0,0.0,0.0,0.0,2500.0,0.0,40732.0,1.0,0.0,0.0,36404.0,750.0,0.0,2417.0,452.0,609.0,5417.0,63.0,880.0,0.0,11705.0,0.0,0.0,2924.0,3849.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3836.0,1.0,0.0,0.0,0.0,0.0,0.0,1717.0,0.0,81942.0,641.0,0.0,200.0,5.0,0.0,2000.0,4963.0,0.0,965.0,28500.0,0.0,0.0,0.0,8952.0,7411.0,35.0,0.0,137862.0,0.0,7000.0,0.0,0.0,3400.0,480.0,6192.0,0.0,117567.0,0.0,30064.0,0.0,82834.0,0.0,0.0,0.0,2471.0,1.0,0.0,0.0,69229.0,105745.0,0.0,2629.0,0.0,0.0,0.0,0.0,0.0,159.0,826301.0,2677971.0,20321.0,372.0,0.0,0.0,0.0,0.0,18555.0,0.0,39.0


In [18]:
# so unfortunately these aren't always zero, however just interpolating these leading NaNs would temper too much with the given data, so let's just set them to zero
df.reset_index(inplace=True)
first_index = df.groupby("location").apply(lambda x: x.iloc[0]["index"])
df.drop(columns = ["index"], inplace = True)
# create list of indices that we want to change
valid_indices = [np.arange(first, last-1) for first, last in zip(first_index, first_valid_index)]
# flatten it to be a 1D array instead of 2D
valid_indices = [elem for sublist in valid_indices for elem in sublist]
# also set the people_vaccinated, people_fully_vaccinated, new_vaccinations to 0 for these rows
df.loc[valid_indices, ["total_vaccinations", "people_vaccinated", "people_fully_vaccinated", "new_vaccinations"]] = 0

In [19]:
# let's look at the percentage of missing values for columns again
print("Percentage of missing values for each column")
(df.isna().sum() / len(df)).to_frame().T

Percentage of missing values for each column


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,new_tests,total_tests,positive_rate,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,0.0,0.0,0.0,0.0,0.0,0.0,0.080843,0.080869,0.098969,0.522387,0.520898,0.4551,0.192474,0.204655,0.228693,0.233132,0.085081,0.0,0.016051,0.037295,0.04825,0.042654,0.037716,0.343801,0.032418,0.016244,0.241819,0.252493,0.510013,0.117656,0.005289,0.02712


In [20]:
# it's looking a lot better now, but the ~50% missing values for new_tests/total_tests are really annoying because these are such interesting columns
# lets's check for leading NaNs
temp = df.copy()
temp.reset_index(inplace=True)
# get the first index for each country
first_index = temp.groupby("location").apply(lambda x: x.iloc[0]["index"])
# get the first valid index for each country for the column total_tests
first_valid_index = temp.groupby("location").apply(lambda x: x["total_tests"].first_valid_index())
print("Number of leading NaNs for each country for the column total_tests")
(first_valid_index - first_index).to_frame().T

Number of leading NaNs for each country for the column total_tests


location,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,Costa Rica,Cote d'Ivoire,Croatia,Cuba,Cyprus,Czechia,Democratic Republic of Congo,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Estonia,Eswatini,Ethiopia,Fiji,Finland,France,Gabon,Gambia,Georgia,Germany,Ghana,Greece,Grenada,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Israel,Italy,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,Kosovo,Kuwait,Kyrgyzstan,Laos,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,Lithuania,Luxembourg,Madagascar,Malawi,Malaysia,Maldives,Mali,Malta,Mauritania,Mauritius,Mexico,Moldova,Monaco,Mongolia,Montenegro,Morocco,Mozambique,Myanmar,Namibia,Nepal,Netherlands,New Zealand,Nicaragua,Niger,Nigeria,North Macedonia,Norway,Oman,Pakistan,Palestine,Panama,Papua New Guinea,Paraguay,Peru,Philippines,Poland,Portugal,Qatar,Romania,Russia,Rwanda,Saint Kitts and Nevis,Saint Lucia,Saint Vincent and the Grenadines,Samoa,San Marino,Sao Tome and Principe,Saudi Arabia,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Slovakia,Slovenia,Solomon Islands,Somalia,South Africa,South Korea,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan,Tajikistan,Tanzania,Thailand,Timor,Togo,Trinidad and Tobago,Tunisia,Turkey,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
0,NaN,0.0,NaN,245.0,NaN,487.0,0.0,8.0,56.0,36.0,423.0,554.0,10.0,0.0,NaN,4.0,26.0,422.0,429.0,17.0,3.0,27.0,4.0,41.0,NaN,34.0,NaN,NaN,478.0,NaN,5.0,NaN,NaN,NaN,31.0,153.0,88.0,NaN,NaN,0.0,33.0,7.0,10.0,18.0,NaN,NaN,0.0,NaN,NaN,17.0,17.0,NaN,17.0,484.0,0.0,NaN,0.0,0.0,29.0,NaN,486.0,495.0,401.0,48.0,3.0,6.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,8.0,2.0,0.0,43.0,14.0,46.0,43.0,18.0,0.0,24.0,13.0,13.0,59.0,0.0,4.0,NaN,438.0,79.0,NaN,421.0,0.0,441.0,NaN,NaN,NaN,80.0,20.0,0.0,39.0,0.0,0.0,8.0,NaN,135.0,18.0,NaN,0.0,435.0,NaN,246.0,NaN,0.0,6.0,8.0,18.0,3.0,17.0,2.0,NaN,NaN,40.0,33.0,35.0,NaN,15.0,NaN,0.0,338.0,2.0,0.0,64.0,55.0,0.0,12.0,15.0,33.0,24.0,474.0,NaN,565.0,NaN,NaN,NaN,0.0,1.0,0.0,NaN,NaN,75.0,0.0,0.0,NaN,NaN,1.0,1.0,144.0,72.0,22.0,NaN,NaN,NaN,88.0,NaN,0.0,NaN,NaN,0.0,478.0,1.0,52.0,393.0,16.0,11.0,37.0,0.0,60.0,38.0,15.0,NaN,NaN,NaN,21.0,NaN,1.0,47.0


In [21]:
# unfortunately this is not so easy to solve
# maybe looking at the percentage of missing values will help us somehow
print("Percentage of non-NaN values for each country for the column total_tests")
(df.groupby("location").count()["total_tests"] / df.groupby("location").size()).to_frame().T

Percentage of non-NaN values for each country for the column total_tests


location,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,Costa Rica,Cote d'Ivoire,Croatia,Cuba,Cyprus,Czechia,Democratic Republic of Congo,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Estonia,Eswatini,Ethiopia,Fiji,Finland,France,Gabon,Gambia,Georgia,Germany,Ghana,Greece,Grenada,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Israel,Italy,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,Kosovo,Kuwait,Kyrgyzstan,Laos,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,Lithuania,Luxembourg,Madagascar,Malawi,Malaysia,Maldives,Mali,Malta,Mauritania,Mauritius,Mexico,Moldova,Monaco,Mongolia,Montenegro,Morocco,Mozambique,Myanmar,Namibia,Nepal,Netherlands,New Zealand,Nicaragua,Niger,Nigeria,North Macedonia,Norway,Oman,Pakistan,Palestine,Panama,Papua New Guinea,Paraguay,Peru,Philippines,Poland,Portugal,Qatar,Romania,Russia,Rwanda,Saint Kitts and Nevis,Saint Lucia,Saint Vincent and the Grenadines,Samoa,San Marino,Sao Tome and Principe,Saudi Arabia,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Slovakia,Slovenia,Solomon Islands,Somalia,South Africa,South Korea,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan,Tajikistan,Tanzania,Thailand,Timor,Togo,Trinidad and Tobago,Tunisia,Turkey,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
0,0.0,0.752053,0.0,0.076299,0.0,0.001653,0.986992,0.648298,0.644172,0.927653,0.244733,0.013289,0.837881,0.936066,0.0,0.337641,0.953344,0.057143,0.001661,0.771242,0.939044,0.719413,0.287415,0.027375,0.0,0.72623,0.0,0.0,0.076805,0.0,0.95092,0.0,0.0,0.0,0.945513,0.003053,0.849673,0.0,0.0,0.986928,0.897858,0.869775,0.775578,0.965461,0.0,0.0,0.991935,0.0,0.0,0.680713,0.922204,0.0,0.627713,0.063018,0.995161,0.0,0.915702,0.741235,0.949153,0.0,0.056291,0.081531,0.127214,0.132104,0.791391,0.89694,0.0,0.985099,0.0,0.0,0.0,0.0,0.0,0.036641,0.921824,0.995153,0.904321,0.74513,0.566879,0.791332,0.966019,0.995208,0.956723,0.662273,0.970992,0.437398,0.738843,0.365289,0.0,0.150662,0.863563,0.0,0.112795,0.99513,0.041534,0.0,0.0,0.0,0.861564,0.956311,0.990291,0.441472,0.437607,0.990812,0.95082,0.0,0.775777,0.206954,0.0,0.993538,0.081967,0.0,0.453947,0.0,0.969156,0.919463,0.820643,0.875828,0.935681,0.13871,0.848142,0.0,0.0,0.411955,0.697262,0.933977,0.0,0.895498,0.0,0.976974,0.011706,0.988525,0.977124,0.895062,0.895765,0.993506,0.951456,0.611916,0.775889,0.773179,0.112985,0.0,0.001656,0.0,0.0,0.0,0.993506,0.991883,0.995098,0.0,0.0,0.123664,0.993464,0.995106,0.0,0.0,0.970636,0.877863,0.242268,0.125387,0.952381,0.0,0.0,0.0,0.85209,0.0,0.99542,0.0,0.0,0.977099,0.040268,0.985294,0.834437,0.210098,0.967051,0.753769,0.770732,0.947612,0.901082,0.932824,0.752066,0.0,0.0,0.0,0.291603,0.0,0.946667,0.908027


In [22]:
# not really sure what to do with these
# we'll do some interpolation later for these (between first and last valid value for each country) and see to what extent that fixes it
# let's look at other columns that have a high percentage of missing values
print("Percentage of non-NaN values for each country for the column positive_rate")
(df.groupby("location").count()["positive_rate"] / df.groupby("location").size()).to_frame().T

Percentage of non-NaN values for each country for the column positive_rate


location,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,Costa Rica,Cote d'Ivoire,Croatia,Cuba,Cyprus,Czechia,Democratic Republic of Congo,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Estonia,Eswatini,Ethiopia,Fiji,Finland,France,Gabon,Gambia,Georgia,Germany,Ghana,Greece,Grenada,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Israel,Italy,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,Kosovo,Kuwait,Kyrgyzstan,Laos,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,Lithuania,Luxembourg,Madagascar,Malawi,Malaysia,Maldives,Mali,Malta,Mauritania,Mauritius,Mexico,Moldova,Monaco,Mongolia,Montenegro,Morocco,Mozambique,Myanmar,Namibia,Nepal,Netherlands,New Zealand,Nicaragua,Niger,Nigeria,North Macedonia,Norway,Oman,Pakistan,Palestine,Panama,Papua New Guinea,Paraguay,Peru,Philippines,Poland,Portugal,Qatar,Romania,Russia,Rwanda,Saint Kitts and Nevis,Saint Lucia,Saint Vincent and the Grenadines,Samoa,San Marino,Sao Tome and Principe,Saudi Arabia,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Slovakia,Slovenia,Solomon Islands,Somalia,South Africa,South Korea,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan,Tajikistan,Tanzania,Thailand,Timor,Togo,Trinidad and Tobago,Tunisia,Turkey,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
0,0.0,0.750411,0.0,0.493506,0.0,0.0,0.986992,0.970827,0.898773,0.44373,0.298217,0.008306,0.967897,0.932787,0.0,0.977383,0.944012,0.268908,0.0,0.784314,0.976936,0.934747,0.894558,0.0,0.0,0.927869,0.0,0.0,0.073733,0.0,0.976994,0.269231,0.0,0.0,0.934295,0.0,0.838235,0.0,0.0,0.977124,0.925865,0.972669,0.764026,0.953947,0.687196,0.624382,0.983871,0.0,0.0,0.941653,0.675851,0.0,0.627713,0.180763,0.987097,0.0,0.923967,0.816361,0.938367,0.811927,0.178808,0.153078,0.330113,0.132104,0.965232,0.974235,0.0,0.976821,0.0,0.0,0.0,0.0,0.0,0.035115,0.980456,0.0,0.91821,0.897727,0.910828,0.913323,0.954693,0.985623,0.94745,0.960461,0.964885,0.518699,0.73719,0.608264,0.0,0.256623,0.857143,0.0,0.180135,0.987013,0.0,0.0,0.0,0.279461,0.861564,0.946602,0.97411,0.812709,0.678632,0.981623,0.970492,0.0,0.762684,0.0,0.0,0.993538,0.254098,0.0,0.491776,0.0,0.987013,0.942953,0.830795,0.951987,0.97856,0.13871,0.980614,0.0,0.0,0.912763,0.863124,0.924316,0.081862,0.959807,0.60522,0.983553,0.083612,0.983607,0.977124,0.884259,0.894137,0.983766,0.0,0.959742,0.933539,0.943709,0.183811,0.0,0.0,0.0,0.0,0.0,0.982143,0.987013,0.986928,0.0,0.0,0.865649,0.98366,0.995106,0.0,0.0,0.986949,0.978626,0.457045,0.851393,0.947773,0.0,0.0,0.727554,0.85209,0.0,0.987786,0.0,0.0,0.969466,0.15604,0.986928,0.849338,0.343648,0.955519,0.822446,0.923577,0.984592,0.890263,0.922137,0.935537,0.0,0.0,0.0,0.532824,0.0,0.978333,0.901338


In [23]:
# this looks similar to total_tests (either countries have a high number of non-NaN values (> 80%) or a low number (< 20%))
# also not sure what to do with these (we'll also interpolate it later)
# let's take a look at another column with a high percentage of NaN-values
print("Percentage of non-NaN values for each country for the column extreme_poverty")
(df.groupby("location").count()["extreme_poverty"] / df.groupby("location").size()).to_frame().T

Percentage of non-NaN values for each country for the column extreme_poverty


location,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,Costa Rica,Cote d'Ivoire,Croatia,Cuba,Cyprus,Czechia,Democratic Republic of Congo,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Estonia,Eswatini,Ethiopia,Fiji,Finland,France,Gabon,Gambia,Georgia,Germany,Ghana,Greece,Grenada,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Israel,Italy,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,Kosovo,Kuwait,Kyrgyzstan,Laos,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,Lithuania,Luxembourg,Madagascar,Malawi,Malaysia,Maldives,Mali,Malta,Mauritania,Mauritius,Mexico,Moldova,Monaco,Mongolia,Montenegro,Morocco,Mozambique,Myanmar,Namibia,Nepal,Netherlands,New Zealand,Nicaragua,Niger,Nigeria,North Macedonia,Norway,Oman,Pakistan,Palestine,Panama,Papua New Guinea,Paraguay,Peru,Philippines,Poland,Portugal,Qatar,Romania,Russia,Rwanda,Saint Kitts and Nevis,Saint Lucia,Saint Vincent and the Grenadines,Samoa,San Marino,Sao Tome and Principe,Saudi Arabia,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Slovakia,Slovenia,Solomon Islands,Somalia,South Africa,South Korea,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan,Tajikistan,Tanzania,Thailand,Timor,Togo,Trinidad and Tobago,Tunisia,Turkey,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0


In [24]:
# ok this is even more extreme now it's either 100% or 0% now, we can't really do anything here
# let's take a look at another column with a high percentage of NaN-values
print("Percentage of non-NaN values for each country for the column handwashing_facilities")
(df.groupby("location").count()["handwashing_facilities"] / df.groupby("location").size()).to_frame().T

Percentage of non-NaN values for each country for the column handwashing_facilities


location,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burundi,Cambodia,Cameroon,Canada,Cape Verde,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo,Costa Rica,Cote d'Ivoire,Croatia,Cuba,Cyprus,Czechia,Democratic Republic of Congo,Denmark,Djibouti,Dominica,Dominican Republic,Ecuador,Egypt,El Salvador,Equatorial Guinea,Estonia,Eswatini,Ethiopia,Fiji,Finland,France,Gabon,Gambia,Georgia,Germany,Ghana,Greece,Grenada,Guatemala,Guinea,Guinea-Bissau,Guyana,Haiti,Honduras,Hong Kong,Hungary,Iceland,India,Indonesia,Iran,Iraq,Ireland,Israel,Italy,Jamaica,Japan,Jordan,Kazakhstan,Kenya,Kiribati,Kosovo,Kuwait,Kyrgyzstan,Laos,Latvia,Lebanon,Lesotho,Liberia,Libya,Liechtenstein,Lithuania,Luxembourg,Madagascar,Malawi,Malaysia,Maldives,Mali,Malta,Mauritania,Mauritius,Mexico,Moldova,Monaco,Mongolia,Montenegro,Morocco,Mozambique,Myanmar,Namibia,Nepal,Netherlands,New Zealand,Nicaragua,Niger,Nigeria,North Macedonia,Norway,Oman,Pakistan,Palestine,Panama,Papua New Guinea,Paraguay,Peru,Philippines,Poland,Portugal,Qatar,Romania,Russia,Rwanda,Saint Kitts and Nevis,Saint Lucia,Saint Vincent and the Grenadines,Samoa,San Marino,Sao Tome and Principe,Saudi Arabia,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Slovakia,Slovenia,Solomon Islands,Somalia,South Africa,South Korea,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan,Tajikistan,Tanzania,Thailand,Timor,Togo,Trinidad and Tobago,Tunisia,Turkey,Uganda,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,Yemen,Zambia,Zimbabwe
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0


In [25]:
# we can't do anything here as well..
# so let's do some interpolation (we'll only interpolate between the first valid value and the last, because it would probably temper too much with the data)
# first we'll look at the percentage of missing values for each column again
print("Percentage of missing values for each column")
cols = df.columns
(df.isna().sum() / len(df)).to_frame().T

Percentage of missing values for each column


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,new_tests,total_tests,positive_rate,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,0.0,0.0,0.0,0.0,0.0,0.0,0.080843,0.080869,0.098969,0.522387,0.520898,0.4551,0.192474,0.204655,0.228693,0.233132,0.085081,0.0,0.016051,0.037295,0.04825,0.042654,0.037716,0.343801,0.032418,0.016244,0.241819,0.252493,0.510013,0.117656,0.005289,0.02712


In [26]:
# since we're only gonna interpolate between the first and last valid value for each country, we can basically put each column in here and see to what extent it fixes something
# we'll only interpolate the total_columns and add the missing values later for the new_columns
cols_to_interpolate = [
    "total_deaths", "reproduction_rate", "total_tests", "positive_rate", "total_vaccinations","people_vaccinated", "people_fully_vaccinated",
    "stringency_index", "population_density", "median_age", "aged_65_older", "aged_70_older","gdp_per_capita", "extreme_poverty",
    "cardiovasc_death_rate", "diabetes_prevalence", "female_smokers", "male_smokers", "handwashing_facilities", "hospital_beds_per_thousand",
    "life_expectancy", "human_development_index"
]
df = df.groupby("location").apply(lambda x: x[df.columns.difference(cols_to_interpolate)].join(x[cols_to_interpolate].interpolate(method = "linear", axis = 0, limit_area = "inside")))[cols]
# let's change new_deaths, new_tests and new_vaccinations accordingly now
df[["new_deaths", "new_tests", "new_vaccinations"]] = df.groupby("location").apply(lambda x: x[["total_deaths", "total_tests", "total_vaccinations"]].diff()).to_numpy()

In [27]:
# let's see how that affected the percentage of missing values for each column
print("Percentage of missing values for each column")
(df.isna().sum() / len(df)).to_frame().T

Percentage of missing values for each column


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,new_tests,total_tests,positive_rate,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,0.0,0.0,0.0,0.0,0.0,0.0,0.080843,0.082463,0.098969,0.42551,0.424363,0.431202,0.007995,0.009501,0.009317,0.009641,0.084319,0.0,0.016051,0.037295,0.04825,0.042654,0.037716,0.343801,0.032418,0.016244,0.241819,0.252493,0.510013,0.117656,0.005289,0.02712


In [28]:
print("Percentage of countries that only have NaN-values for a given column")
(df.groupby("location").apply(lambda x: x.isna().all()).sum(axis = 0) / len(df["location"].unique())).to_frame().T

Percentage of countries that only have NaN-values for a given column


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,new_tests,total_tests,positive_rate,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,0.0,0.0,0.0,0.0,0.0,0.0,0.015957,0.015957,0.026596,0.319149,0.303191,0.308511,0.0,0.0,0.0,0.0,0.074468,0.0,0.015957,0.037234,0.047872,0.042553,0.037234,0.345745,0.031915,0.015957,0.244681,0.255319,0.505319,0.12234,0.005319,0.026596


In [29]:
# it might be fine to use the average value of the continent for a country for a specific column if it's completly missing 
# (this only applies to some columns (columns that describe local factors and that we won't expect to change much over the time interval))
# however we should probably first look at the standard deviation and compare it with the mean and if the std is too big, we can't do it for that column
cols_to_consider = [
    "location", "population_density", "median_age", "aged_65_older", "aged_70_older", "extreme_poverty", 
    "cardiovasc_death_rate", "diabetes_prevalence", "female_smokers", "male_smokers", "handwashing_facilities",
    "hospital_beds_per_thousand", "life_expectancy", "human_development_index"
]
# get the continent for each country
continents = df.groupby("location").apply(lambda x: x.iloc[0]["continent"])
# get the mean value for the considered columns for each country
temp = df[cols_to_consider].groupby("location").mean()
# add continent as column
temp["continent"] = continents
# now get the relative size of std to mean for each continent and then average that out for all continents
means = temp.groupby("continent").mean()
means_all = (temp.groupby("continent").std() / temp.groupby("continent").mean()).mean()
print("Relative magnitude of std compared to mean (= 1 -> mean and std are the same)")
means_all.to_frame().T

Relative magnitude of std compared to mean (= 1 -> mean and std are the same)


,population_density,median_age,aged_65_older,aged_70_older,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,1.89741,0.181294,0.468467,0.510914,1.134216,0.418043,0.399842,0.797155,0.422045,0.341071,0.626664,0.06479,0.136037


In [30]:
# for some columns the std is quite big compared to the mean value, for these columns it's probably not a good idea to just use the mean
# now we just need to define a threshhold at which we want to use the mean of the continent for missing values
threshhold = 0.5
cols_to_use_mean = means_all <= threshhold
cols_to_use_mean = cols_to_use_mean[cols_to_use_mean].index
print("Columns to use average value of the continent for missing values with threshhold = {}:".format(threshhold))
cols_to_use_mean.to_list()

Columns to use average value of the continent for missing values with threshhold = 0.5:


['median_age',
 'aged_65_older',
 'cardiovasc_death_rate',
 'diabetes_prevalence',
 'male_smokers',
 'handwashing_facilities',
 'life_expectancy',
 'human_development_index']

In [31]:
print("Mean values that will be used for NaN cells")
means[cols_to_use_mean]

Mean values that will be used for NaN cells


,median_age,aged_65_older,cardiovasc_death_rate,diabetes_prevalence,male_smokers,handwashing_facilities,life_expectancy,human_development_index
continent,,,,,,,,
Africa,21.111321,3.670453,286.848132,5.621509,27.882857,25.155273,64.072264,0.562692
Asia,30.470213,6.584711,290.287261,9.329111,39.697368,71.172920,74.596809,0.739891
Europe,42.070000,17.719625,230.159415,6.558409,35.592500,93.954000,79.824545,0.879881
North America,31.661905,8.828762,200.113045,10.733043,22.308333,77.541308,75.411739,0.755957
Oceania,27.012500,7.282500,375.077000,13.482500,35.542857,30.549500,72.853750,0.711750
South America,30.216667,8.277250,187.323917,7.810833,24.662500,65.990667,75.090833,0.763167


In [32]:
for col in cols_to_use_mean:
    # get all rows that are NaN for this column
    nan_indices = df[col].isna()
    # set it to the mean value of the continent of that country for that column
    df.loc[nan_indices, col] = means.loc[df[nan_indices]["continent"], col].to_numpy()

In [33]:
# let's look at the missing values for each column again
print("Percentage of missing values for each column")
(df.isna().sum() / len(df)).to_frame().T

Percentage of missing values for each column


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,reproduction_rate,new_tests,total_tests,positive_rate,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,0.0,0.0,0.0,0.0,0.0,0.0,0.080843,0.082463,0.098969,0.42551,0.424363,0.431202,0.007995,0.009501,0.009317,0.009641,0.084319,0.0,0.016051,0.0,0.0,0.042654,0.037716,0.343801,0.0,0.0,0.241819,0.0,0.0,0.117656,0.0,0.0


In [34]:
# except a few columns it's looking decent now, not sure what to do further

In [ ]:
import dash
from dash import dcc
from dash import html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
import json

app = dash.Dash(__name__)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

fig = px.bar(df, x="location", y="new_cases", color="continent", barmode="group")

fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(
        children='Overview of covid 19',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),

    html.Div(children='Historical data of covid 19 cases.', style={
        'textAlign': 'center',
        'color': colors['text']
    }),

    dcc.Graph(
        id='clientside-graph-px',
        figure=fig
    ),
    
     dcc.Store(
        id='clientside-figure-store-px'
    ),
    
# select y-parameter
    
    'Indicator',
    dcc.Dropdown(
        id='clientside-graph-indicator-px',
        options=[
            {'label': 'New Cases', 'value': 'new_cases'},
            {'label': 'Total Cases', 'value': 'total_cases'},
            {'label': 'Date', 'value': 'date'}
        ],
        value='new_cases'
    ),
    
# select x-parameter 
    
    'Country',
    dcc.Dropdown(
        id='clientside-graph-country-px',
        options=[
            {'label': 'New Cases', 'value': 'new_cases'},
            {'label': 'Total Cases', 'value': 'total_cases'},
            {'label': 'Location', 'value': 'location'}
        ],
        value='location'
    ), 
    
    html.Hr(),
    html.Details([
        html.Summary('Contents of figure storage'),
        dcc.Markdown(
            id='clientside-figure-json-px'
        )
    ])
])

@app.callback(
    Output('clientside-figure-store-px', 'data'),
    Input('clientside-graph-indicator-px', 'value'),
    Input('clientside-graph-country-px', 'value')
)
def update_store_data(indicator, country):
    #dff = df[df['location'] == country]
    return [{
        'x': df[indicator],
        'y': df[country],
        'mode': 'markers'
    }]


app.clientside_callback(
    """
    function(figure, scale) {
        if(figure === undefined) {
            return {'data': [], 'layout': {}};
        }
        const fig = Object.assign({}, figure, {
            'layout': {
                ...figure.layout,
                'yaxis': {
                    ...figure.layout.yaxis, type: scale
                }
             }
        });
        return fig;
    }
    """,
    Output('clientside-graph-px', 'figure'),
    Input('clientside-figure-store-px', 'data'),
    Input('clientside-graph-scale-px', 'value')
)


@app.callback(
    Output('clientside-figure-json-px', 'children'),
    Input('clientside-figure-store-px', 'data')
)
def generated_px_figure_json(data):
    return '```\n'+json.dumps(data, indent=2)+'\n```'

if __name__ == '__main__':
    app.run_server()
    

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Nov/2021 17:02:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2021 17:02:57] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2021 17:02:57] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2021 17:02:57] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Nov/2021 17:02:57] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Nov/2021 17:02:57] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Nov/2021 17:02:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov/2021 17:02:57] "GET /_dash-component-suites/dash/dcc/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Nov/2021 17:02:57] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Nov/2021 17:02:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Nov